In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
# IN and Out variables for data 
dimension = 20
fps = 100 #frames per second
filters = 128

x = tf.placeholder(tf.float32, [None, dimension, fps])  #Inputs
y = tf.placeholder(tf.float32, [None, 2])   #Labels
keep_prob = tf.placeholder(dtype= tf.float32)

In [3]:
input_x = tf.reshape( x, shape = [-1, dimension, fps, 1])

conv1 = tf.layers.conv2d( input_x, 
                                filters = 128,
                                kernel_size = [20, 3],
                                padding = 'VALID',
                                name = "conv1_caps")
print(conv1)

max_out1 = tf.contrib.layers.maxout( conv1, 
                                  num_units = 64,
                                  axis = -1,
                                  scope = "max_out1")
print(max_out1)

Tensor("conv1_caps/BiasAdd:0", shape=(?, 1, 98, 128), dtype=float32)
Tensor("max_out1/Max:0", shape=(?, 1, 98, 64), dtype=float32)


In [4]:
conv2 = tf.layers.conv2d( max_out1, 
                                filters = 128,
                                kernel_size = [1, 3],
                                padding = 'VALID',
                                name = "conv2_caps")
print(conv2)

max_out2 = tf.contrib.layers.maxout( conv2,
                                   num_units = 64,
                                   axis = -1,
                                   scope = "max_out2")
print(max_out2)

Tensor("conv2_caps/BiasAdd:0", shape=(?, 1, 96, 128), dtype=float32)
Tensor("max_out2/Max:0", shape=(?, 1, 96, 64), dtype=float32)


In [5]:
conv3 = tf.layers.conv2d( max_out2,
                                filters = 128,
                                kernel_size = [1, 3],
                                padding = 'VALID',
                                name = "conv3_caps")
print(conv3)



max_out3 = tf.contrib.layers.maxout(conv3, 
                                   num_units = 64,
                                   axis = -1,
                                   scope = "max_out3")
print(max_out3)

Tensor("conv3_caps/BiasAdd:0", shape=(?, 1, 94, 128), dtype=float32)
Tensor("max_out3/Max:0", shape=(?, 1, 94, 64), dtype=float32)


In [6]:
conv4 = tf.layers.conv2d( max_out3, 
                                filters = 128,
                                kernel_size = [1,2],
                                strides = 2,
                                padding = "VALID",
                                name = "conv4_caps")
# epsilon = 1e-7
# conv_out

max_out4 = tf.contrib.layers.maxout( conv4,
                                           num_units = 64,
                                           axis = -1,
                                           scope = "max_out4")
print(conv4)
print(max_out4)

Tensor("conv4_caps/BiasAdd:0", shape=(?, 1, 47, 128), dtype=float32)
Tensor("max_out4/Max:0", shape=(?, 1, 47, 64), dtype=float32)


In [7]:
conv5 = tf.layers.conv2d( max_out4, 
                                    filters = 128,
                                    kernel_size = [1, 3],
                                    padding = "VALID",
                                    name = "con5_caps")
print(conv5)

max_out5 = tf.contrib.layers.maxout( conv5,
                                           num_units = 64,
                                           axis = -1,
                                           scope = "max_out5")
print(max_out5)

Tensor("con5_caps/BiasAdd:0", shape=(?, 1, 45, 128), dtype=float32)
Tensor("max_out5/Max:0", shape=(?, 1, 45, 64), dtype=float32)


In [8]:
conv6 = tf.layers.conv2d( max_out5, 
                                 filters = 128,
                                 kernel_size = [1, 3],
                                 padding = "VALID",
                                 name = "conv6_caps")
print(conv6)

max_out6 = tf.contrib.layers.maxout(conv6,
                                    num_units = 64,
                                    axis = -1,
                                    scope = "max_out6")
print(max_out6)

Tensor("conv6_caps/BiasAdd:0", shape=(?, 1, 43, 128), dtype=float32)
Tensor("max_out6/Max:0", shape=(?, 1, 43, 64), dtype=float32)


In [9]:
conv7 = tf.layers.conv2d( max_out6, 
                                 filters = 128,
                                 kernel_size = [1, 2],
                                 strides = [1, 2],
                                 padding = "VALID",
                                 name = "conv7_caps")
print(conv7)

max_out7 = tf.contrib.layers.maxout(conv7,
                                    num_units = 64,
                                    axis = -1,
                                    scope = "max_out7")
print(max_out7)

Tensor("conv7_caps/BiasAdd:0", shape=(?, 1, 21, 128), dtype=float32)
Tensor("max_out7/Max:0", shape=(?, 1, 21, 64), dtype=float32)


In [10]:
weights_conv10 = tf.get_variable( "weights_conv10", shape = [ 1, 1, 64, 2],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_conv10 = tf.get_variable("bias_conv10", shape = [2],
                            initializer = tf.contrib.layers.xavier_initializer())

conv10 = tf.nn.conv2d( max_out7, weights_conv10, strides = [1, 1, 1, 1], padding = "VALID")
conv10_out = tf.nn.relu( conv10 + bias_conv10 )
conv10_drop = tf.nn.dropout( conv10_out, keep_prob)
conv10_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 1, 21, 2) dtype=float32>

In [11]:
#Final output layer
weights_final = tf.get_variable( "weights_final", shape = [ 1, 17, 2, 2],
                               initializer = tf.contrib.layers.xavier_initializer())
bias_final = tf.get_variable("bias_final", shape = [2],
                            initializer = tf.contrib.layers.xavier_initializer())

conv_final = tf.nn.conv2d( conv10_drop, weights_final, strides = [1, 1, 17, 1], padding = "VALID")
conv_final = tf.nn.relu( conv_final + bias_final )
conv_final

<tf.Tensor 'Relu_1:0' shape=(?, 1, 1, 2) dtype=float32>

In [12]:
final_output = tf.reshape(conv_final, shape = [-1, 2])

output = tf.nn.softmax(final_output)

In [13]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = final_output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [14]:
#initializing tensors using sessoion
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)

features_path = "E:\\ShubhamMGDCC\\overlapData_20D"
development_path = "E:\\ShubhamMGDCC\\20D\\devoverlapData_20D"
#evaluation_path = "E:\\Hemlata_CSL\\MFCC\\CNN_input_testing_eval"
model_path = "E:\\ShubhamMGDCC\\20D\\100_50_full_conv5_3k_maxout_2\\Models\\"
    
dev_sp_path ="E:\\ShubhamMGDCC\\20D\\100_50_full_conv5_3k_maxout_2\\dev"
#eva_sp_path = "E:\\Hemlata_CSL\\MFCC\\4_conv\\eval"

In [15]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [16]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = len(list(os.listdir(test_path)))//2
    
    for folders in range(1,indexs+1):
        features = np.load(os.path.join(test_path, "features_"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels_"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(output, feed_dict = {x:features, keep_prob:1.0})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [17]:
print("start")
time.sleep(20)
print("complete")

start
complete


In [ ]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,50):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ x:features[:1024], y:labels[:1024], keep_prob:1.0})
        train_step.run(session = session,feed_dict={ x:features[1024:], y:labels[1024:], keep_prob:1.0})
        
    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()
    save_path = saver.save(session, model_path +str(i))
    

iteration   1
FRR   0.0  FAR    1.0    mid    0.0863766074180603
 develop eer   0.0

INFO:tensorflow:E:\ShubhamMGDCC\20D\100_50_full_conv5_3k_maxout_2\Models\0 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
FRR   0.0  FAR    1.0    mid    0.05072063207626343
 develop eer   0.0

INFO:tensorflow:E:\ShubhamMGDCC\20D\100_50_full_conv5_3k_maxout_2\Models\1 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3


In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config=config) as session:
    for i in range(49,80):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_dev = get_EER(development_path, (i+1), dev_sp_path)
        print(" develop eer  ",eer_dev)
        print()
        save_path = saver.save(session, model_path +str(i+1))